In [ ]:
import numpy as np
import time

# —————————————
# 1) 상수 및 좌표 변환
# —————————————
MAX_PLAYER = 1  # Black
MIN_PLAYER = 2  # White
DIRECTIONS = [(-1,-1),(-1,0),(-1,1),(0,-1),(0,1),(1,-1),(1,0),(1,1)]
SIDE_DIRS = [(-1,0),(1,0),(0,-1),(0,1)]

POSITION_WEIGHTS = np.array([
    [100,  -40,   -15,    5,    5,   -15,  -40,  100], # Increased negative weight for positions next to corners
    [-40,  -60,   -5,   -5,   -5,   -5,  -60,  -40], # Increased negative weight for positions next to corners
    [-15,   -5,   -1,   -1,   -1,   -1,   -5,  -15], # Decreased weight for edge positions
    [5,     -5,   -1,   -1,   -1,   -1,   -5,    5],
    [5,     -5,   -1,   -1,   -1,   -1,   -5,    5],
    [-15,   -5,   -1,   -1,   -1,   -1,   -5,  -15], # Decreased weight for edge positions
    [-40,  -60,   -5,   -5,   -5,   -5,  -60,  -40], # Increased negative weight for positions next to corners
    [100,  -40,   -15,    5,    5,   -15,  -40,  100], # Increased negative weight for positions next to corners
])
WEIGHTS = {
    'positional':     1,
    'disc_diff':      0,
    'mobility':       78,
    'pot_mobility':   74,
    'corner':         801,
    'stability':      380,
    'edge_stability': 202,
    'frontier':       10,
    'line_completion_penalty': -100 # New weight for penalizing line completion
}

# 좌표 변환 유틸리티
def col_to_index(c: str) -> int:
    return ord(c.upper()) - ord('A')

def row_to_index(r: str) -> int:
    return int(r) - 1

def coord_to_index(coord: str) -> tuple[int,int]:
    if len(coord) != 2:
        raise ValueError(f"좌표 형식 오류: {coord}")
    return row_to_index(coord[1]), col_to_index(coord[0])

def index_to_coord(pos: tuple[int,int]) -> str:
    return f"{chr(pos[1]+ord('A'))}{pos[0]+1}"

# —————————————
# 2) 초기 보드 생성
# —————————————
def create_initial_board() -> np.ndarray:
    b = np.zeros((8,8), int)
    for coord, p in [("E4", MAX_PLAYER), ("D5", MAX_PLAYER), ("D4", MIN_PLAYER), ("E5", MIN_PLAYER)]:
        r,c = coord_to_index(coord)
        b[r,c] = p
    return b

# —————————————
# 3) 게임 로직: 이동 생성, 적용, 종료 검사
# —————————————
def get_legal_moves(board: np.ndarray, player: int) -> list[tuple[int,int]]:
    moves = []
    opp = 3 - player
    for r in range(8):
        for c in range(8):
            if board[r, c] != 0:
                continue

            for dr, dc in DIRECTIONS:
                nr, nc = r + dr, c + dc
                flips = []

                if 0 <= nr < 8 and 0 <= nc < 8 and board[nr, nc] == opp:
                    flips.append((nr, nc))
                    nr += dr
                    nc += dc

                    while 0 <= nr < 8 and 0 <= nc < 8 and board[nr, nc] == opp:
                        flips.append((nr, nc))
                        nr += dr
                        nc += dc

                    if 0 <= nr < 8 and 0 <= nc < 8 and board[nr, nc] == player:
                        if flips:
                            moves.append((r, c))
                            break
    return sorted(list(set(moves)))


def apply_move(board: np.ndarray, move: tuple[int,int], player: int) -> np.ndarray:
    b2 = board.copy(); r,c = move; b2[r,c] = player; opp=3-player
    for dr,dc in DIRECTIONS:
        flips=[]; nr,nc=r+dr, c+dc
        while 0<=nr<8 and 0<=nc<8 and b2[nr,nc]==opp:
            flips.append((nr,nc)); nr+=dr; nc+=dc
        if 0<=nr<8 and 0<=nc<8 and b2[nr,nc]==player:
            for rr,cc in flips: b2[rr,cc]=player
    return b2


def is_terminal(board: np.ndarray) -> bool:
    if np.all(board != 0):
        return True
    if not get_legal_moves(board, MAX_PLAYER) and not get_legal_moves(board, MIN_PLAYER):
        return True
    return False

# —————————————
# 4) 평가 함수 헬퍼
# —————————————

def phase_weighted_disc_diff(board, player):
    total = np.count_nonzero(board)
    phase = total / 60.0
    my   = np.count_nonzero(board == player)
    opp  = np.count_nonzero(board == 3 - player)
    return (my - opp) * phase

def positional_value(board, player):
    opp     = 3 - player
    my_mask = (board == player).astype(int)
    op_mask = (board == opp).astype(int)
    return int((my_mask * POSITION_WEIGHTS).sum() -
               (op_mask * POSITION_WEIGHTS).sum())

def count_stable_discs(board, player):
    stable  = np.zeros_like(board, bool)
    corners = [(0,0),(0,7),(7,0),(7,7)]
    for r0,c0 in corners:
        if board[r0,c0] != player: continue
        stable[r0,c0] = True
        for dr,dc in DIRECTIONS:
            r,c = r0+dr, c0+dc
            while 0 <= r < 8 and 0 <= c < 8 and board[r,c] == player:
                stable[r,c] = True
                r += dr; c += dc
    return np.count_nonzero(stable)

def potential_mobility(board, player):
    opp = 3 - player
    pot = 0
    for r in range(8):
        for c in range(8):
            if board[r,c] != opp: continue
            for dr,dc in DIRECTIONS:
                nr,nc = r+dr, c+dc
                if 0 <= nr < 8 and 0 <= nc < 8 and board[nr,nc] == 0:
                    pot += 1
    return pot

def frontier_discs(board, player):
    cnt = 0
    for r in range(8):
        for c in range(8):
            if board[r,c] != player: continue
            for dr,dc in DIRECTIONS:
                nr,nc = r+dr, c+dc
                if 0 <= nr < 8 and 0 <= nc < 8 and board[nr,nc] == 0:
                    cnt += 1
                    break
    return cnt

def count_edge_stable_discs(board, player):
    stable  = np.zeros_like(board, bool)
    corners = [(0,0),(0,7),(7,0),(7,7)]
    for r0,c0 in corners:
        if board[r0,c0] != player: continue
        for dr,dc in SIDE_DIRS:
            r,c = r0, c0
            while True:
                r += dr; c += dc
                if not (0 <= r < 8 and 0 <= c < 8): break
                if board[r,c] != player: break
                stable[r,c] = True
    edge_positions = (
        [(0,i) for i in range(8)] + [(7,i) for i in range(8)] +
        [(i,0) for i in range(8)] + [(i,7) for i in range(8)]
    )
    return sum(1 for (r,c) in edge_positions
               if stable[r,c] and board[r,c] == player)

# New helper function to check for line completion risk
def check_line_completion_risk(board, move, player):
    r, c = move
    opp = 3 - player
    risk = 0
    # Check in all directions
    for dr, dc in DIRECTIONS:
        line_of_opponents = False
        nr, nc = r + dr, c + dc
        # Check if the adjacent disc is opponent's
        if 0 <= nr < 8 and 0 <= nc < 8 and board[nr, nc] == opp:
            line_of_opponents = True
            # Continue in the same direction to see if the line is terminated by opponent's disc
            step = 2
            while True:
                rr, cc = r + dr * step, c + dc * step
                if not (0 <= rr < 8 and 0 <= cc < 8):
                    break # Out of bounds
                if board[rr, cc] == 0:
                    break # Empty square, line not terminated
                if board[rr, cc] == player:
                    # Terminated by player's disc, this move is valid and flips discs
                    break
                if board[rr, cc] == opp:
                    # Terminated by opponent's disc - this is the risky scenario
                    risk += 1 # Increment risk for this direction
                    break
                step += 1
    return risk


# —————————————
# 5) 종합 평가 함수
# —————————————
def evaluate(board, player):
    opp = 3 - player
    F1 = phase_weighted_disc_diff(board, player)
    F2 = len(get_legal_moves(board, player)) - len(get_legal_moves(board, opp))
    F3 = sum(board[r,c]==player for r,c in [(0,0),(0,7),(7,0),(7,7)]) - \
         sum(board[r,c]==opp    for r,c in [(0,0),(0,7),(7,0),(7,7)])
    F4 = count_stable_discs(board, player) - count_stable_discs(board, opp)
    F5 = potential_mobility(board, player) - potential_mobility(board, opp)
    F6 = frontier_discs(board, opp) - frontier_discs(board, player)
    F7 = count_edge_stable_discs(board, player) - count_edge_stable_discs(board, opp)
    F8 = positional_value(board, player)

    # Calculate line completion penalty
    # We need the potential moves to check the risk for each move
    # This evaluation function is called for a given board state, not a specific move.
    # The line completion penalty should be calculated within the move evaluation, not here.
    # For now, we will remove the line completion penalty from the main evaluate function
    # and consider how to incorporate it into the move evaluation within alphabeta or find_best_move.

    return (
        WEIGHTS['disc_diff']      * F1 +
        WEIGHTS['mobility']       * F2 +
        WEIGHTS['corner']         * F3 +
        WEIGHTS['stability']      * F4 +
        WEIGHTS['pot_mobility']   * F5 +
        WEIGHTS['frontier']       * F6 +
        WEIGHTS['edge_stability'] * F7 +
        WEIGHTS['positional']     * F8
    )

# —————————————
# 6) 상대 프론티어 최소화 프리필터
# —————————————
def filter_moves_by_opp_frontier(board: np.ndarray, player: int) -> list[tuple[int,int]]:
    opp = 3 - player
    moves = get_legal_moves(board, player)
    if not moves:
        return []
    base = frontier_discs(board, opp)
    reductions = [(base - frontier_discs(apply_move(board, m, player), opp)) for m in moves]
    max_red = max(reductions)
    return [m for m, r in zip(moves, reductions) if r == max_red]

# —————————————
# 7) Minimax + α-β + Iterative Deepening
# —————————————
def alphabeta(board: np.ndarray, depth: int, α: float, β: float, player: int) -> float:
    if depth == 0 or is_terminal(board):
        return evaluate(board, player)

    moves = filter_moves_by_opp_frontier(board, player) or get_legal_moves(board, player)

    if not moves:
        # If no legal moves, evaluate the current board state for the current player
        # after a pass. This is a simplified approach.
        # A more robust implementation might check if the *other* player can move.
        return evaluate(board, player)

    if player == MAX_PLAYER:
        value = -np.inf
        for m in moves:
            child = apply_move(board, m, player)
            # Calculate penalty for this specific move
            penalty = check_line_completion_risk(board, m, player) * WEIGHTS['line_completion_penalty']
            move_value = alphabeta(child, depth-1, α, β, MIN_PLAYER) + penalty # Add penalty to the evaluation
            value = max(value, move_value)
            α = max(α, value)
            if α >= β:
                break
        return value
    else:
        value = np.inf
        for m in moves:
            child = apply_move(board, m, player)
            # Calculate penalty for this specific move
            # Penalty should be from the perspective of the player making the move (MIN_PLAYER here)
            penalty = check_line_completion_risk(board, m, MIN_PLAYER) * WEIGHTS['line_completion_penalty']
            move_value = alphabeta(child, depth-1, α, β, MAX_PLAYER) + penalty # Add penalty to the evaluation
            value = min(value, move_value)
            β = min(β, value)
            if β <= α:
                break
        return value

def find_best_move(board: np.ndarray, player: int, max_time: float = 1.0) -> tuple[int,int] | None:
    start = time.time()
    best_move = None
    depth = 1

    while time.time() - start < max_time:
        current_best_move = None
        best_score = -np.inf if player == MAX_PLAYER else np.inf
        moves = get_legal_moves(board, player)

        if not moves:
            return None

        # Sorting moves can sometimes improve alpha-beta pruning efficiency,
        # but it's omitted here for simplicity.

        for m in moves:
            if time.time() - start > max_time:
                break

            child = apply_move(board, m, player)
            # Note: We call alphabeta for the *next* player (3-player) with depth-1
            score = alphabeta(child, depth - 1, -np.inf, np.inf, 3 - player)

            if (player == MAX_PLAYER and score > best_score) or (player == MIN_PLAYER and score < best_score):
                best_score = score
                current_best_move = m

        if current_best_move is not None:
             best_move = current_best_move

        depth += 1

    return best_move

# game_state_history and redo_stack are already globally initialized
game_state_history = [] # 게임 상태 저장을 위한 리스트
redo_stack = [] # 되돌린 상태 저장을 위한 리스트 (앞으로 가기 기능용)


# —————————————
# 8) Main: 교대로 추천 및 입력 처리 (Undo/Redo 포함)
# —————————————
def main():
    # game_state_history와 redo_stack은 이미 전역으로 초기화됨

    board = create_initial_board()
    current = MAX_PLAYER

    # 초기 상태 저장
    game_state_history.append((board.copy(), current))

    print("초기 보드:")
    print("  A B C D E F G H")
    for i,row in enumerate(board): print(i+1, ' '.join(str(x) for x in row))
    while True:
        turn = 'Black' if current==MAX_PLAYER else 'White'

        # 게임 종료 조건 확인
        if is_terminal(board):
            print("게임 종료!")
            break

        # 추천 출력
        rec = find_best_move(board, current)
        if rec:
            print(f"추천 {turn} Move: {index_to_coord(rec)}")
        else:
            # 둘 곳이 없는 경우 패스
            print(f"{turn} 둘 곳 없음. 건너뜁니다.")
            current = 3 - current
            # 상대방도 둘 곳 없으면 게임 종료
            if is_terminal(board):
                print("양쪽 모두 둘 곳이 없어 게임 종료!")
                break
            continue # 턴을 넘겼으므로 다시 루프 시작

        # 사용자 입력
        move_str = input(f"{turn} 입력 (A1~H8, 'undo', 'redo', 'exit' 종료): ").strip().lower()

        if move_str == 'exit':
            break
        elif move_str == 'undo':
            if len(game_state_history) > 1:
                # 현재 상태를 redo 스택에 저장
                redo_stack.append((board.copy(), current))
                # 이전 상태 불러오기
                prev_board, prev_current = game_state_history.pop()
                board, current = prev_board, prev_current
                print("이전 상태로 되돌립니다.")
                # 되돌린 후 보드 출력
                print("  A B C D E F G H")
                for i, row in enumerate(board): print(i+1, ' '.join(str(x) for x in row))
            else:
                print("더 이상 되돌릴 수 없습니다.")
            continue
        elif move_str == 'redo':
            if redo_stack:
                # redo 스택에서 상태 불러오기
                next_board, next_current = redo_stack.pop()
                # 현재 상태를 history에 저장 (redo 후 다시 undo할 경우를 대비)
                game_state_history.append((board.copy(), current))
                board, current = next_board, next_current
                print("앞으로 이동합니다.")
                # 이동 후 보드 출력
                print("  A B C D E F G H")
                for i, row in enumerate(board): print(i+1, ' '.join(str(x) for x in row))
            else:
                print("더 이상 앞으로 갈 수 없습니다.")
            continue

        try:
            pos = coord_to_index(move_str)
        except ValueError as e:
            print(f"잘못된 형식 또는 값입니다: {e}"); continue

        if pos not in get_legal_moves(board, current):
            print("유효하지 않은 수입니다."); continue

        # 유효한 수인 경우 상태 저장 및 착수 적용
        game_state_history.append((board.copy(), current)) # 현재 상태 저장
        redo_stack.clear() # 새로운 착수 시 redo 스택 초기화

        board = apply_move(board, pos, current)
        current = 3 - current

        # 보드 출력
        print("  A B C D E F G H")
        for i,row in enumerate(board): print(i+1, ' '.join(str(x) for x in row))

    # 게임 종료 집계
    b_cnt = np.count_nonzero(board==MAX_PLAYER)
    w_cnt = np.count_nonzero(board==MIN_PLAYER)
    print(f"게임 종료. Black: {b_cnt}, White: {w_cnt}")

if __name__=="__main__": main()